<a href="https://colab.research.google.com/github/2001abdulrehman/FYP/blob/main2/catract_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.lite.python import interpreter as interpreter_wrapper
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Extract the data from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/New Model dataset/train.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset-tflite')

# Define paths to the training data
train_dir = 'dataset-tflite'

# Create an image data generator
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Build the CNN model
model = keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3),
           data_format="channels_last"),  # Input layer with Conv2D
    MaxPooling2D(2, 2),  # Max-pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second Conv2D layer
    MaxPooling2D(2, 2),  # Second Max-pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Third Conv2D layer
    MaxPooling2D(2, 2),  # Third Max-pooling layer
    Flatten(),  # Flatten layer
    Dense(512, activation='relu'),  # Dense (fully connected) layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    verbose=1
)

# Save the trained model
model.save('cataract_detection_model.h5')

Found 1180 images belonging to 2 classes.
Epoch 1/20
37/37 [==============================] - 101s 3s/step - loss: 0.7856 - accuracy: 0.6593
Epoch 2/20
37/37 [==============================] - 100s 3s/step - loss: 0.4224 - accuracy: 0.8195
Epoch 3/20
37/37 [==============================] - 99s 3s/step - loss: 0.3797 - accuracy: 0.8347
Epoch 4/20
37/37 [==============================] - 104s 3s/step - loss: 0.4196 - accuracy: 0.8110
Epoch 5/20
37/37 [==============================] - 101s 3s/step - loss: 0.3515 - accuracy: 0.8424
Epoch 6/20
37/37 [==============================] - 98s 3s/step - loss: 0.2975 - accuracy: 0.8720
Epoch 7/20
37/37 [==============================] - 99s 3s/step - loss: 0.2927 - accuracy: 0.8822
Epoch 8/20
37/37 [==============================] - 100s 3s/step - loss: 0.2798 - accuracy: 0.8797
Epoch 9/20
37/37 [==============================] - 105s 3s/step - loss: 0.2582 - accuracy: 0.9051
Epoch 10/20
37/37 [==============================] - 102s 3s/step - lo

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**QUANTIZE AND CONVERT TO TFLITE**

In [ ]:


# Save the trained model to an h5 file
model.save('/content/cataract_detection_model.h5')

# Load the trained model
loaded_model = tf.keras.models.load_model('/content/cataract_detection_model.h5')

# Convert the model to a TFLite quantized model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply optimizations including quantization
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model to a file
with open('/content/quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)


Model converted and saved to your_model.tflite


**TESTING**

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/Catracts/Test.zip', 'r') as zip_ref:
    zip_ref.extractall('test_dataset')

In [ ]:

# Define the path to the extracted test dataset directory
test_dir = '/content/test_dataset'
#/content/drive/MyDrive/Catracts/your_model.tflite
# Load the TensorFlow Lite model
tflite_model_path = '/content/drive/MyDrive/Catracts/quantized_model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create an image data generator for test data
test_data_generator = ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,  # Set batch_size to 1 to handle images one by one
    class_mode='binary',
    shuffle=False  # Ensure no shuffling for evaluation
)

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test set to get predictions
for i in range(len(test_generator)):
    image, label = test_generator[i]

    # Preprocess the image to match the input shape expected by the model
    image = img_to_array(image[0])  # Convert from PIL image to numpy array
    image = np.expand_dims(image, axis=0)  # Add a batch dimension

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    tflite_predictions = interpreter.get_tensor(output_details[0]['index'])
    predicted_labels.append(tflite_predictions[0][0])
    true_labels.append(label[0])

# Convert the lists to numpy arrays for comparison
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate accuracy
threshold = 0.5  # Threshold for binary classification
predicted_classes = (predicted_labels >= threshold).astype('int32')
correct_predictions = np.sum(predicted_classes == true_labels)
accuracy = correct_predictions / len(true_labels)

print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

Found 212 images belonging to 2 classes.
Accuracy on the test set: 79.72%


**RANDOM TEST**

In [ ]:

# Load the TFLite model
tflite_path = '/content/drive/MyDrive/Catracts/quantized_model.tflite'
interpreter = interpreter_wrapper.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Define the path to the folder containing mixed cataract and normal images
image_folder = '/content/drive/MyDrive/Catracts/RANDOM'

# Create empty lists to store cataract and normal image filenames
cataract_images = []
normal_images = []

# Iterate over the images in the folder and classify them
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)

        # Load the image
        image = Image.open(image_path)
        image = image.resize((150, 150))  # Resize to match the model's input size
        image = tf.convert_to_tensor(image, dtype=tf.float32) / 255.0  # Normalize the image
        image = image.numpy()
        image = image[np.newaxis, ...]  # Add a batch dimension

        # Set input tensor
        input_details = interpreter.get_input_details()
        interpreter.set_tensor(input_details[0]['index'], image)

        # Invoke the interpreter
        interpreter.invoke()

        # Get the output tensor
        output_details = interpreter.get_output_details()
        output = interpreter.get_tensor(output_details[0]['index'])

        # Classify the image
        is_cataract = output[0][0] < 0.5  # Adjust the threshold as needed

        # Separate cataract and normal images
        if is_cataract:
            cataract_images.append(filename)
        else:
            normal_images.append(filename)

# Display the cataract and normal image filenames
print("Cataract Images:")
for filename in cataract_images:
    print(filename)

print("\nNormal Images:")
for filename in normal_images:
    print(filename)


Cataract Images:
B (3).png
B (2).png
B (4).png
B (1).png
B (5).png

Normal Images:
A (4).png
A (3).png
A (2).png
A (1).png
A (5).png


**Single Image Test**

In [ ]:

def classify_single_image(image_path, tflite_model_path):
    # Load the TFLite model
    interpreter = interpreter_wrapper.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Load the image
    image = Image.open(image_path)
    image = image.resize((150, 150))  # Resize to match the model's input size
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add a batch dimension

    # Set input tensor
    input_details = interpreter.get_input_details()
    interpreter.set_tensor(input_details[0]['index'], image)

    # Invoke the interpreter
    interpreter.invoke()

    # Get the output tensor
    output_details = interpreter.get_output_details()
    output = interpreter.get_tensor(output_details[0]['index'])

    # Classify the image
    is_cataract = output[0][0] < 0.5  # Adjust the threshold as needed

    # Return classification result
    return "Cataract" if is_cataract else "Normal"

#  path to your single image
image_to_classify = '/content/f0229fdf-f0fd-4c8b-a5c2-7eb4fa4f4681.jpg'

#  path to your TFLite model
model_path = '/content/drive/MyDrive/Catracts/quantized_model.tflite'

result = classify_single_image(image_to_classify, model_path)
print(f"The image is classified as: {result}")


The image is classified as: Normal
